In [1]:
from google.colab import files
uploaded=files.upload()

Saving archive.zip to archive.zip


In [2]:
import zipfile
import os

In [3]:
with zipfile.ZipFile('archive.zip','r') as zip:
  zip.extractall("unzipped_data")
print(os.listdir("unzipped_data"))

['train', 'test']


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Flatten,GlobalAveragePooling2D,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

In [5]:
train_dir = '/content/unzipped_data/train'
test_dir = '/content/unzipped_data/test'

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)
test_generator=test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [6]:
base_model=ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
base_model.trainable=True
for layer in base_model.layers[:100]:
  layer.trainable=False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dropout(0.3)(x)
x=Dense(128,activation="relu")(x)
x=Dropout(0.2)(x)
predictions=Dense(train_generator.num_classes,activation="softmax")(x)
model=Model(inputs=base_model.input,outputs=predictions)

In [8]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=3, verbose=1)
]


In [9]:
history=model.fit(train_generator,validation_data=test_generator,epochs=30,callbacks=[callbacks])

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
898/898 ━━━━━━━━━━━━━━━━━━━━ 447s 459ms/step - accuracy: 0.4097 - loss: 1.5633 - val_accuracy: 0.5075 - val_loss: 1.4864 - learning_rate: 0.0010
Epoch 2/30
898/898 ━━━━━━━━━━━━━━━━━━━━ 372s 414ms/step - accuracy: 0.5585 - loss: 1.1831 - val_accuracy: 0.5878 - val_loss: 1.1212 - learning_rate: 0.0010
Epoch 3/30
898/898 ━━━━━━━━━━━━━━━━━━━━ 370s 412ms/step - accuracy: 0.5940 - loss: 1.0919 - val_accuracy: 0.5864 - val_loss: 1.1060 - learning_rate: 0.0010
Epoch 4/30
898/898 ━━━━━━━━━━━━━━━━━━━━ 372s 414ms/step - accuracy: 0.6106 - loss: 1.0529 - val_accuracy: 0.6154 - val_loss: 1.0475 - learning_rate: 0.0010
Epoch 5/30
898/898 ━━━━━━━━━━━━━━━━━━━━ 374s 416ms/step - accuracy: 0.6241 - loss: 1.0151 - val_accuracy: 0.6205 - val_loss: 1.0178 - learning_rate: 0.0010
Epoch 6/30
898/898 ━━━━━━━━━━━━━━━━━━━━ 374s 416ms/step - accuracy: 0.6298 - loss: 0.9901 - val_accuracy: 0.6152 - val_loss: 1.0253 - learning_rate: 0.0010
Epoch 7/30
898/898 ━━━━━━━━━━━━━━━━━━━━ 387s 422ms/step - accura

In [ ]:
files.download('emotional_resnet.h5')